In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.optim as optim

In [ ]:
def factorize(A, k, device=torch.device("cpu"), epsilon = 0.001):
	"""Factorize the matrix D into A and B"""
	A = A.to(device)
	# Randomly initialize A and B
	W = torch.randn(A.shape[0], k, requires_grad=True, device=device)
	H = torch.randn(k, A.shape[1], requires_grad=True, device=device)
	# Optimizer
	optimizer = optim.Adam([W, H], lr=0.01)
	mask = ~torch.isnan(A)
	epoch = 0
	# Train the model
	while True:
		# Compute the loss
		diff_matrix = torch.mm(W, H) - A
		diff_vector = diff_matrix[mask]
		loss = torch.norm(diff_vector)

		# Zero the gradients
		optimizer.zero_grad()

		# Backpropagate
		loss.backward()

		# Update the parameters
		optimizer.step()
		epoch += 1
		if epoch>1:
			if prev_loss-loss < epsilon:
				break
		# if epoch>=100 and epoch%100==0:
		#     print(f'Epoch:{epoch}--->Loss:{loss}, prev_loss:{prev_loss}')
		prev_loss = loss

	# print(f'No of Epochs:{epoch}')

	return W, H, loss

def reconstruct_img(masked_image,factors=50,device=torch.device("cpu")):#returns torch
	W, H, _ = factorize(masked_image,factors,device)
	reconstruct_img = torch.mm(W, H).cpu().detach()
	return reconstruct_img


def factorize_3d(A, k, device=torch.device("cpu"), epsilon = 0.001):
	A = A.to(device)
	A1,A2,A3 = A[:,:,0],A[:,:,1],A[:,:,2]
	# Randomly initialize A and B
	A_3d = [A1,A2,A3]
	W_3d = []
	H_3d = []
	loss_3d = []

	for a in A_3d:
		W,H,loss = factorize(a,k,device,epsilon)
		W_3d.append(W)
		H_3d.append(H)
		loss_3d.append(loss)

	return W_3d,H_3d,loss_3d

def reconstruct_3d_img(masked_image,factors=50,device=torch.device("cpu")):
	W_3d,H_3d,_ = factorize_3d(masked_image,factors,device)
	for i in range(3):
		a = torch.mm(W_3d[i], H_3d[i]).cpu().detach()
		A.append(a)
